In [1]:
import sys
sys.path.append('../../src/')

from lib.pysoarlib import *
import os

curdir = os.path.abspath('')

Soar exposes a python binding for ease of use. `pysoarlib` is a package created by `amininger`. You can find it here: https://github.com/amininger/pysoarlib. I have tweaked it a bit to work better in this container.

We can create an agent using the SoarAgent class.

In [51]:
rule_agent = SoarAgent(
    agent_name="hello world",
    write_to_stdout=True,
    spawn_debugger=False,
    agent_source=curdir + "/1_hello_world.soar",
)

--------- SOURCING PRODUCTIONS ------------
Total: 1 production sourced.


Next we can add a connector to see the stdout

In [52]:
rule_agent.add_connector("simple", AgentConnector(rule_agent))
rule_agent.connect()
result = rule_agent.execute_command("run")

print(result)

[DEBUG] Run event handler 13
Hello World
System halted.
Interrupt received.This Agent halted.
--> 1 decision cycle executed. 1 rule fired. 
An agent halted during the run.


killing the agent will destory and remove the kernel

In [53]:
rule_agent.kill()

In the default config file are the following 

## 3.2
#### Hello world operator

This time, we load in from a config file. The file contains the following settings:

`write-to-stdout = true`

How much detail to print when sourcing files: none, summary, or full

`source_output="summary"`

In [54]:
# this time, load in from a config file. it populates the fields
# 'write_to_stdout' for us
operator_agent = SoarAgent(
    agent_source="1_hello_world_operator.soar",
    config_filename='default.config',
    root_dir=curdir,
)

--------- SOURCING PRODUCTIONS ------------
Total: 2 productions sourced.


In [55]:
operator_agent.add_connector("hw-operator", AgentConnector(operator_agent))
operator_agent.connect()
result = operator_agent.execute_command("run")

print(result)

[DEBUG] Run event handler 13
1:    O: O1 (hello-world)
Hello World
System halted.
Interrupt received.This Agent halted.
--> 1 decision cycle executed. 2 rules fired. 
An agent halted during the run.


With `print`, we can view all the attributes an dvalues that have `s1` as their identifier.

In [56]:
operator_agent.execute_command("print s1", print_res=True);

print s1
(S1 ^epmem E1 ^io I1 ^operator O1 ^operator O1 + ^reward-link R1 ^smem L1
       ^superstate nil ^type state)


The attributes `^io`, `^superstate`, and `^type` are created automatically before the program runs.
The 'operator' attribute is created when the hello-world operator (o1) is selected.

Printing at a depth of 2 (we abbreviate `print` to `p`)

In [57]:
operator_agent.execute_command("p s1 -d 2", print_res=True);

p s1 -d 2
(S1 ^epmem E1 ^io I1 ^operator O1 ^operator O1 + ^reward-link R1 ^smem L1
       ^superstate nil ^type state)
  (E1 ^command C1 ^present-id 1 ^result R2)
  (I1 ^input-link I2 ^output-link I3)
  (O1 ^name hello-world)
  (L1 ^command C2 ^result R3)


View the augmentation of `o1`

In [58]:
operator_agent.execute_command("p o1", print_res=True);

p o1
(O1 ^name hello-world)


#### View the augmentation of `o1`

TODO: define augmentation

There are two attributes: ^input-link and ^output-link

`input-link`: where an agent's sensory information is available in WM

`output-link`: where action commands must be created for the agent to move in its world

In [59]:
operator_agent.execute_command("p o1", print_res=True);

p o1
(O1 ^name hello-world)


####  View the augmentation of `I1`

There are two attributes: `^input-link` and `^output-link`

`input-link`: where an agent's sensory information is available in WM

`output-link`: where action commands must be created for the agent to move in its world

In [60]:
operator_agent.execute_command("p I1", print_res=True);

p I1
(I1 ^input-link I2 ^output-link I3)


## Custom agent

In [7]:
agent_raw = """
sp {propose*hello-world
   (state <s> ^type state)
-->
   (<s> ^operator <o> +)
   (<o> ^name hello-world)
}

sp {apply*hello-world
   (state <s> ^operator <o>)
   (<o> ^name hello-world)
-->
   (write |Hello World|)
   (halt)
}
"""
    
custom_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=curdir + '/default.config',
)
a

--------- SOURCING PRODUCTIONS ------------
Total: 2 productions sourced.
[DEBUG] Run event handler 13
1:    O: O1 (hello-world)
Hello World
System halted.
Interrupt received.This Agent halted.
